In [1]:
import pandas as pd
import numpy as np

- **長期選股**

##### 三地開發交易量太少，因此選競國

In [2]:
def calculate_average_growth_rate(data, indicator):
    data['date'] = pd.to_datetime(data['date'])
    data['quarter'] = data['date'].dt.to_period(freq='Q')
    
    pivot_table = data.pivot_table(index=['stock_name', 'stock_symbol'], columns='quarter', values=indicator, aggfunc='mean')
    quarterly_growth_rate = pivot_table.pct_change(axis=1)
    filtered_quarters = ['2022Q1', '2023Q4']
    filtered_data = quarterly_growth_rate.drop(columns=filtered_quarters)
    filtered_data = filtered_data.replace([np.inf, -np.inf], np.nan).dropna()
    average_growth_rate = filtered_data.mean(axis=1)
    sorted_data = average_growth_rate.sort_values(ascending=False)
    sorted_dataframe = sorted_data.to_frame(name='average_growth_rate')
    return sorted_dataframe

selected_columns = ['stock_name', 'stock_symbol', 'return_on_equity', 'eps', 'date']
data = pd.read_csv("bda2024_微股力_財報數據-2年.csv", usecols=selected_columns)

roe_sorted = calculate_average_growth_rate(data, 'return_on_equity')
eps_sorted = calculate_average_growth_rate(data, 'eps')

merged_df = pd.merge(roe_sorted, eps_sorted, left_index=True, right_index=True, how='inner')
merged_df.head(10)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17132\673948508.py:6: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  quarterly_growth_rate = pivot_table.pct_change(axis=1)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17132\673948508.py:6: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  quarterly_growth_rate = pivot_table.pct_change(axis=1)


,,average_growth_rate_x,average_growth_rate_y
stock_name,stock_symbol,,
三地開發,1438,48.268618,44.209084
競國,6108,31.975323,13.289481
景岳,3164,24.632848,1.599028
基泰,2538,21.587304,1.304536
竣邦-KY,4442,16.913304,27.248015
遠百,2903,14.144869,-4.984510
精英,2331,12.548984,12.796487
神盾,6462,11.665851,19.564262
彰源,2030,11.609297,10.477941


- **短期投資**

In [3]:
import pandas as pd

def calculate_average_investment_trust(data_file, stock_data_file):
    data = pd.read_csv(data_file)
    stock_df = pd.read_csv(stock_data_file)
    selected_columns = ['stock_name', 'stock_symbol','investment_trust_bought', 'investment_trust_sold', 'date']
    data = data[selected_columns]
    data['date'] = pd.to_datetime(data['date']).dt.date

    last_five_days = data.groupby(['stock_name', 'stock_symbol']).apply(lambda x: x.sort_values('date').tail(5)).reset_index(drop=True)
    last_five_days_stock = stock_df.groupby(['stock_name', 'stock_symbol']).apply(lambda x: x.sort_values('date').tail(5)).reset_index(drop=True)
    last_five_days['net_investment_trust'] = last_five_days['investment_trust_bought'] - last_five_days['investment_trust_sold']

    avg_net_investment_trust = last_five_days.groupby(['stock_name', 'stock_symbol'])['net_investment_trust'].mean()
    avg_volume = last_five_days_stock.groupby(['stock_name', 'stock_symbol'])['volume'].mean()
    avg_df = pd.concat([pd.DataFrame(avg_net_investment_trust), pd.DataFrame(avg_volume)], axis=1)
    sorted_data = avg_df.sort_values(by='net_investment_trust', ascending=False)

    return sorted_data.head(10)

result = calculate_average_investment_trust("bda2024_微股力_籌碼數據-2年.csv", "bda2024_微股力_個股交易數據-2年.csv")
result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17132\992121625.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_file)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17132\992121625.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  stock_df = pd.read_csv(stock_data_file)


,,net_investment_trust,volume
stock_name,stock_symbol,,
宏碁,2353,11049600.0,60202.6
永豐餘,1907,4781200.0,34333.4
聯電,2303,3997402.0,73962.4
良維,6290,1618200.0,26744.4
長榮,2603,1585721.8,30063.2
聚陽,1477,1480800.0,NaN
群創,3481,1290571.0,211077.0
鴻海,2317,1120279.4,47792.4
臻鼎-KY,4958,1111714.8,5113.4
